<a href="https://colab.research.google.com/github/Christinathattil/MACHINE-LEARNING-PROJECTS/blob/Main/2348511_lab_3__apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing The Necessary Libraries"""

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from collections import Counter

In [ ]:
#importing the dataset
data=pd.read_csv("/content/Market_Basket_Optimisation.csv")
data

In [ ]:
"""#Data Preprocessing"""

#Checking for null values
data.isnull().sum()

In [ ]:
#dropping the olive column as it has no values
data=data.drop(['olive oil'],axis=1)

In [ ]:
#Storing the data in a list for each row excluding the Nan values
transactions=[]
for i in range(0,data.shape[0]):
  transaction=[str(data.values[i,j]) for j in  range(0,data.shape[1]) if str(data.values[i,j]) !='nan' ]
  transactions.append(transaction)

transactions

In [ ]:
#Finding the types of items in the dataset
unique_items=set()
for transaction in transactions:
    unique_items.update(item for item in transaction)

In [ ]:
unique_list_items=list(unique_items)

unique_list_items

In [ ]:
#Total number of present in dataset
len(unique_list_items)

In [ ]:
#Converting the dataset into binary matrix(1-items present 0 - no item )
binary_matrix = pd.DataFrame(columns=unique_list_items)

new_data=[]

for transaction in transactions:
    row = [1 if item in transaction else 0 for item in unique_list_items]
    new_data.append(row)


binary_matrix = pd.concat([binary_matrix,pd.DataFrame(new_data,columns=unique_list_items)],ignore_index= True)
print(binary_matrix)

In [ ]:
"""#EDA"""

#Finding the total number of each item
item_counts= binary_matrix.sum()
sorted_products = item_counts.sort_values(ascending=False)

#Bar plot- Represent the number of times each item has been bought
plt.figure(figsize=(16,8))
plt.bar(sorted_products.index, sorted_products.values)
plt.title("Product Popularity")
plt.xlabel("Products")
plt.ylabel("Total Count")
plt.xticks(rotation=90)

plt.show()

In [ ]:
"""#Inference"""

The above graph displays the frequency of each product's purchases.The graph helps us to identify the top selling products
in the store and from the graph we can observe that Mineral water is the top selling product with over 1750+ purchases
followed by eggs(2nd top selling product) with over 1250+ purchases.We can also observe that products such as napkins ,
water spray and asparagus are the least bought products.

In [ ]:
#Association Rule Mining"""

frequent_itemset = apriori(binary_matrix,min_support=0.02,use_colnames=True)
rules = association_rules(frequent_itemset,metric='lift',min_threshold=1)

rules.head()


In [ ]:
#Pie chart
plt.figure(figsize=(24, 12))
plt.pie(frequent_itemset['support'], labels=frequent_itemset['itemsets'].apply(lambda x: ', '.join(list(x))), autopct='%1.1f%%', startangle=140)
plt.title('Distribution of Common Itemsets')
plt.axis('equal')
plt.show()

In [ ]:
rules[(rules['confidence']>0.2) & (rules['lift']>1)]

best_choices=rules.sort_values(by='lift',ascending=False)
print('Best Choices would be')
print(best_choices[['antecedents','consequents','lift']])

In [ ]:
worst_choices=rules.sort_values(by='lift',ascending=True)
print('worst Choices would be')
print(worst_choices[['antecedents','consequents','lift']])

In [ ]:
#heat map
heatmap_data = rules.pivot(index='antecedents', columns='consequents', values=['support', 'confidence'])

In [ ]:
plt.figure(figsize=(20, 10))
sns.heatmap(heatmap_data, annot=True, fmt='.2f', cmap='viridis', linewidths=0.5)
plt.title('Association Rules Strength Heatmap')
plt.show()